In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

from tfyolo3 import YoloV3, losses
from tfyolo3.dataloaders import YoloDatasetSingleFile
from tfyolo3.dataloaders import common
from tfyolo3.helpers import draw
from pathlib import Path
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# This notebook is used to understand how the yolo loss works

The first create some random anchors and use the default masks

In [2]:
x = np.arange(5, 46, 5)
anchors = np.array(list(zip(x,x)), dtype=np.float32)
anchors[:,1] += np.random.randint(0, 10, 9)
anchors

array([[ 5., 14.],
       [10., 15.],
       [15., 16.],
       [20., 28.],
       [25., 32.],
       [30., 36.],
       [35., 36.],
       [40., 47.],
       [45., 48.]], dtype=float32)

In [3]:
masks = YoloV3.default_masks
masks

array([[6, 7, 8],
       [3, 4, 5],
       [0, 1, 2]])

## Download a toy dataset

make the following cell executable to download the toy dataset

And load the dataset using the SequenceDataset
---

In [ ]:
filepath = Path('./toy_dataset/data_annotations_train.txt')
target_shape = (256, 256, 3)
batch_size = 2
gridlen = 8
is_training = True
max_objects = 10


train_seq = YoloDatasetSingleFile(filepath,
                                  target_shape,
                                  max_objects,
                                  batch_size, 
                                  anchors,
                                  masks,
                                  is_training=True)

In [ ]:
train_seq.classes

Now we take a batch from the dataset
--

In [ ]:
x_true, y_true_grids = train_seq[0]

The batch contains:
- 2 images

In [ ]:
x_true.shape

- 3 grids

In [ ]:
for i in range(len(y_true_grids)): 
    print(i, '-->', y_true_grids[i].shape, target_shape[0] / y_true_grids[i].shape[1])

The third value plotted represents the size in number of pixel of grid cells

Check that the dataset transformed is correct
----

In [ ]:
for img_idx in range(len(x_true)):
    print('Show annotations for image', img_idx)
    img = x_true[img_idx]

    for i in range(len(y_true_grids)):
        y_data_grid_img = y_true_grids[i][img_idx]
        
        ax = draw.show_img(img)
        grid_len = y_data_grid_img.shape[1]
        draw.grid(ax, img.shape[:2], grid_len)

        grid_cell_size = target_shape[1] / grid_len
        
        for grid_y, grid_x, box in np.argwhere(np.sum(y_data_grid_img[..., :4], axis=-1) > 0):
            box_xyxy = (y_data_grid_img[grid_y,grid_x,box, :4] * target_shape[0]).astype(int)
            class_id = np.argwhere(y_data_grid_img[grid_y,grid_x,box, 5:])[0][0]
            draw.rect(ax, box_xyxy, 'white', 1)

            rect_resp = np.array([grid_x, grid_y]) * grid_cell_size
            rect_resp = np.concatenate([rect_resp, rect_resp + grid_cell_size])
            draw.rect(ax, rect_resp, 'blue', 2)

            draw.point(ax, common.to_center_width_height(box_xyxy)[:2])
        plt.show()

Create the model

In [ ]:
model = YoloV3(target_shape, max_objects, anchors=anchors, num_classes=train_seq.num_classes, training=True)

In [ ]:
tf.keras.utils.plot_model(model.model, show_shapes=True)

## Evaluate how the loss works

We consider two cases:

1. when we got an initialized network the predictions should be around 0.5 (max entropy)
1. when we give as prediction the right labels the loss should be close to zero

In [ ]:
y_pred_grids = model(x_true)
for y_pred in y_pred_grids:
    print(y_pred.shape)

We take i=0 since all the images are in the first grid

In [ ]:
i = 0
y_true = y_true_grids[i]
y_pred = y_pred_grids[i]
anchors_masks = anchors[masks[i]]
img_size = target_shape[0]
loss_fn = losses.make_loss(train_seq.num_classes, anchors, masks, img_size)
ignore_threshold = 0.7

### First Case

In [ ]:
from tfyolo3.losses import Loss

In [ ]:
# 1. transform all pred outputs
# y_pred: (batch_size, grid, grid, anchors, (x, y, w, h, obj, ...cls))
anchors_masks_scaled = anchors_masks / img_size
pred_xyxy, pred_obj, pred_class, pred_xywh = losses.process_predictions(
            tf.cast(y_pred, tf.float32), train_seq.num_classes, anchors_masks_scaled
)
pred_xy = pred_xywh[..., 0:2]
pred_wh = pred_xywh[..., 2:4]

We expect that considering the variable `pred_xywh` the predictions should be:
- for xy in in average 0.5
- for wh close to 0
- for xy1, xy2 close to 0.5

While considering `pred_xyxy` it should be around 0.5

In [ ]:
print('average xy', tf.reduce_mean(pred_xy))
print('average hw', tf.reduce_mean(pred_wh))
print('average xyxy', tf.reduce_mean(pred_xyxy))

This is valid for all the objecteness and classes

In [ ]:
print('average pred_obj', tf.reduce_mean(pred_obj))
print('average pred_class', tf.reduce_mean(pred_class))

In [ ]:
# 2. transform all true outputs
# y_true: (batch_size, grid, grid, anchors, (x, y, w, h, obj, ...cls))
true_box_xyxy, true_obj, true_class = tf.split(
    y_true, (4, 1, train_seq.num_classes), axis=-1)
true_xy = (true_box_xyxy[..., 0:2] + true_box_xyxy[..., 2:4]) / 2
true_wh = true_box_xyxy[..., 2:4] - true_box_xyxy[..., 0:2]

In [ ]:
box_loss_scale = 2 - true_wh[..., 0] * true_wh[..., 1]

3. inverting the pred box equations, to make it comparable with the transformations done for the predictions

In [ ]:
grid_size = tf.shape(y_true)[1]
grid = tf.meshgrid(tf.range(grid_size), tf.range(grid_size))
grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)
true_xy = true_xy * tf.cast(grid_size, tf.float32) - \
    tf.cast(grid, tf.float32)

true_wh = tf.math.log(true_wh / anchors_masks_scaled)
true_wh = tf.where(tf.math.is_inf(true_wh),
                   tf.zeros_like(true_wh), true_wh)

The line 8 contains the opposite transformation made for the predictions

```
box_wh = tf.exp(box_wh) * anchors_masks
```

The masks are used to:
1. separate the boxes that contain objects and should be considered in the objects loss
2. from the boxes that not contain objects and should be considered in the **no object loss**

In [ ]:
# 4. calculate all masks
obj_mask = tf.squeeze(true_obj, -1)
# ignore false positive when iou is over threshold
true_box_mask = tf.boolean_mask(
    true_box_xyxy, tf.cast(obj_mask, tf.bool))
best_iou = tf.reduce_max(Loss.broadcast_iou(
    pred_xyxy, true_box_mask), axis=-1)
ignore_mask = tf.cast(best_iou < ignore_threshold, tf.float32)

Compute all the losses

- xy, wh only with respect the objects that contains elements

In [ ]:
xy_loss = obj_mask * box_loss_scale * \
    tf.reduce_sum(tf.square(true_xy - pred_xy), axis=-1)
wh_loss = obj_mask * box_loss_scale * \
    tf.reduce_sum(tf.square(true_wh - pred_wh), axis=-1)

- the object and no object loss

You can check the whenever the loss is different to zero in the `obj_loss` is zero in the `no_obj_loss` and vice-versa

In [ ]:
obj_cross_entropy = tf.keras.metrics.binary_crossentropy(
    true_obj, pred_obj, from_logits=False)
obj_loss = obj_mask * obj_cross_entropy
no_obj_loss = (1 - obj_mask) * ignore_mask * obj_cross_entropy

- The class loss is computed only for the cells the contains objects

In [ ]:
class_loss = obj_mask * tf.keras.metrics.binary_crossentropy(
            true_class, pred_class, from_logits=False)

- everything is reduced to one value per image

In [ ]:
xy_loss = tf.reduce_sum(xy_loss, axis=(1, 2, 3))
wh_loss = tf.reduce_sum(wh_loss, axis=(1, 2, 3))
obj_loss = tf.reduce_sum(obj_loss, axis=(1, 2, 3))
no_obj_loss = tf.reduce_sum(no_obj_loss, axis=(1, 2, 3))
class_loss = tf.reduce_sum(class_loss, axis=(1, 2, 3))

loss = xy_loss + wh_loss + obj_loss + no_obj_loss + class_loss

loss

### Second Case

- y_true == y_pred

In [ ]:
i = 0
y_true = y_true_grids[i]
y_pred = y_true_grids[i]

- Remember that y_pred is in format xy_min xy_max

In [ ]:
pred_xyxy, pred_obj, pred_class = tf.split(
        y_pred, (4, 1, train_seq.num_classes), axis=-1)

pred_xy = (pred_xyxy[..., 0:2] + pred_xyxy[..., 2:4]) / 2
pred_wh = pred_xyxy[..., 2:4] - pred_xyxy[..., 0:2]

pred_xywh = tf.concat((pred_xy, pred_wh), axis=-1)

In [ ]:
true_box_xyxy, true_obj, true_class = tf.split(
            y_true, (4, 1, train_seq.num_classes), axis=-1)
true_xy = (true_box_xyxy[..., 0:2] + true_box_xyxy[..., 2:4]) / 2
true_wh = true_box_xyxy[..., 2:4] - true_box_xyxy[..., 0:2]

In [ ]:
box_loss_scale = 2 - true_wh[..., 0] * true_wh[..., 1]

# 4. calculate all masks
obj_mask = tf.squeeze(true_obj, -1)
# ignore false positive when iou is over threshold
true_box_mask = tf.boolean_mask(
    true_box_xyxy, tf.cast(obj_mask, tf.bool))
best_iou = tf.reduce_max(Loss.broadcast_iou(
    pred_xyxy, true_box_mask), axis=-1)
ignore_mask = tf.cast(best_iou < ignore_threshold, tf.float32)

In [ ]:
# 5. compute all the losses
xy_loss = obj_mask * box_loss_scale * \
    tf.reduce_sum(tf.square(true_xy - pred_xy), axis=-1)
wh_loss = obj_mask * box_loss_scale * \
    tf.reduce_sum(tf.square(true_wh - pred_wh), axis=-1)

In [ ]:
obj_cross_entropy = tf.keras.metrics.binary_crossentropy(
    true_obj, pred_obj, from_logits=False)
obj_loss = obj_mask * obj_cross_entropy
no_obj_loss = (1 - obj_mask) * ignore_mask * obj_cross_entropy

class_loss = obj_mask * tf.keras.metrics.binary_crossentropy(
    true_class, pred_class, from_logits=False)

In [ ]:
xy_loss = tf.reduce_sum(xy_loss, axis=(1, 2, 3))
wh_loss = tf.reduce_sum(wh_loss, axis=(1, 2, 3))
obj_loss = tf.reduce_sum(obj_loss, axis=(1, 2, 3))
no_obj_loss = tf.reduce_sum(no_obj_loss, axis=(1, 2, 3))
class_loss = tf.reduce_sum(class_loss, axis=(1, 2, 3))

loss = xy_loss + wh_loss + obj_loss + no_obj_loss + class_loss
loss

The loss is 0 when the prediction is equal to the true values

## Conclusion

we have verified that the loss:
- return max entropy value when the network is initialized, and
- return 0 when the y_pred is equal to y_true